In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re
import json
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt

In [2]:
# file naming variables
today = date.today()
last_version = today - timedelta(days=7)
location = '/Users/Sim/Documents/Other/Programming/Personal Projects/Property Price Monitor'

In [3]:
df2 = pd.DataFrame(
    {"Date": [],
     "Average Price": [],
     "Median Price": [],
     "10th Percentile": [],
     "90th Percentile": [],
     "Std dev": [],
     "Sample Size": []
    })

In [4]:
# First import the scraping functions
from ipynb.fs.full.Monitor_funcs import scrape_results_page

### Scraper

*The scraper Loops through rightmove results pages of 5 London areas extracting link, price and featured property status of each property*


**What areas?**

5 relevant, non-overlapping areas defined by a 1 mile (or 0.5 mile) radius around the following tube stations (also see screenshot below):
- Kentish Town (1 mile)
- Royal Oak (1 mile)
- Finchley Road (1 mile)
- Angel (1 mile)
- Mornington Crescent (0.5 miles)


**What search criteria are used?**

I specify that properties must:
- have exactly 2 bedrooms and be a house/flat/apartment to ensure we compare like with like
- have been posted in last 7 days to ensure they don't appear in last week's scrape


**Why are there two scrapers?**

The first scraper loops through the 4 stations with 1mile radii. Due to the much lower number of properties in 0.5 miles around Mornington Crescent, results rarely go onto a second page meaning. Therefore I simply created a specific scraping code for it.

![alt text](Areas.png "Areas")

In [5]:
links = scrape_results_page()[1]
date_time = scrape_results_page()[2]
prices = scrape_results_page()[3]
featured = scrape_results_page()[4]

In [6]:
date_time = pd.to_datetime(date_time,dayfirst = True, format = "%d/%m/%Y %H:%M")
dates = date_time.date

In [7]:
# convert to dataframe
data = {"Links": links,
        "DateTimeScraped": date_time,
        "DateScraped": dates,
        "Price": prices,
        "Featured": featured,
       }
df = pd.DataFrame.from_dict(data)

In [8]:
# Remove featured properties
df = df[df.Featured != 1]

# Sampling 120 rows
df=df.sample(n = 120)

In [9]:
# Key scrape info
date, avg_price, n = df.loc[1,'DateScraped'], round(np.mean(df['Price']),2), len(df)
median_price, ten  = round(np.median(df['Price']),2), round(np.percentile(df['Price'], 10),2)
ninety, st_dev = round(np.percentile(df['Price'], 90),2), round(np.std(df['Price']),3)

# Key info df
df2.loc[len(df2)] = [date, avg_price,median_price, ten,ninety, st_dev,n]
df2

,Date,Average Price,Median Price,10th Percentile,90th Percentile,Std dev,Sample Size
0,2022-11-05,859297.89,850000.0,522500.0,1250000.0,301294.541,120.0


In [10]:
# Save to csv
df.to_csv(f'{location}/full_df_{today}.csv')
df2.to_csv(f'{location}/df_{today}.csv')